In [ ]:
%load_ext autoreload
%autoreload 2

import os
import glob
from mair import notebook_utils
from mair import papers_processing_utils

from typing import List
from tqdm import tqdm
from spacy.lang.en import English
import pandas as pd

tqdm.pandas()

In [ ]:
INPUT_FOLDER = "../data/arxiv_dump/raw_extracted_texts/"
OUTPUT_FOLDER = "../data/arxiv_dump/cleaned_texts/"

MIN_PAPER_LEN = 100
input_files_pattern = os.path.join(INPUT_FOLDER, "*.txt")

# Loading texts

In [ ]:
df = notebook_utils.load_texts_as_df(input_files_pattern)

In [ ]:
df

### Removing empty papers

In [ ]:
df = df[~(df['text'].str.len()<MIN_PAPER_LEN)]
len(df)

# Cleaning

In [ ]:
df['cleaned_text']=df.text.progress_apply(papers_processing_utils.clean_text)

### Finding beginning of paper content

In [ ]:
en = English()
df['doc'] = df.cleaned_text.progress_apply(en)

In [ ]:
# Not ideal, need to find better solution
BEGINNING_WORDS = ['abstract', 'summary']#, 'introduction']
df['beginning'] = df['doc'].apply(find_first, args=(BEGINNING_WORDS,))
df['clean_stripped_text'] = df.progress_apply(lambda row: row.doc[row.beginning:].text, axis=1)

## Saving

In [ ]:
notebook_utils.save_texts_series_to_dir(df['clean_stripped_text'], OUTPUT_FOLDER)